<a href="https://colab.research.google.com/github/Kuper994/TML-project/blob/main/TML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Env Setup

In [ ]:
# !pip install -U torchtext==0.18.0


In [ ]:
from google.colab import drive
import sys
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Trutworthy-ML/Project')
sys.path.append(os.path.abspath('/content/drive/MyDrive/Trutworthy-ML/Project'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import random
import pickle


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Model llama

In [ ]:
from transformers.pipelines.text_generation import ReturnType, Chat
from typing import Union, Sequence, Any


class OurPipeline(transformers.pipelines.TextGenerationPipeline):
  def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

  def _forward(self, model_inputs, **generate_kwargs):
        input_ids = model_inputs["input_ids"]
        attention_mask = model_inputs.get("attention_mask", None)
        # Allow empty prompts
        if input_ids.shape[1] == 0:
            input_ids = None
            attention_mask = None
            in_b = 1
        else:
            in_b = input_ids.shape[0]
        prompt_text = model_inputs.pop("prompt_text")

        # If there is a prefix, we may need to adjust the generation length. Do so without permanently modifying
        # generate_kwargs, as some of the parameterization may come from the initialization of the pipeline.
        prefix_length = generate_kwargs.pop("prefix_length", 0)
        if prefix_length > 0:
            has_max_new_tokens = "max_new_tokens" in generate_kwargs or (
                "generation_config" in generate_kwargs
                and generate_kwargs["generation_config"].max_new_tokens is not None
            )
            if not has_max_new_tokens:
                generate_kwargs["max_length"] = generate_kwargs.get("max_length") or self.model.config.max_length
                generate_kwargs["max_length"] += prefix_length
            has_min_new_tokens = "min_new_tokens" in generate_kwargs or (
                "generation_config" in generate_kwargs
                and generate_kwargs["generation_config"].min_new_tokens is not None
            )
            if not has_min_new_tokens and "min_length" in generate_kwargs:
                generate_kwargs["min_length"] += prefix_length

        # BS x SL
        generated_sequence = self.model.generate(input_ids=input_ids, attention_mask=attention_mask, output_scores=True, return_dict_in_generate=True, **generate_kwargs)
        # print(generated_sequence)
        # forward_res = self.model(input_ids=input_ids, attention_mask=attention_mask)
        # out_b = generated_sequence.shape[0]
        # if self.framework == "pt":
        #     generated_sequence = generated_sequence.reshape(in_b, out_b // in_b, *generated_sequence.shape[1:])
        # elif self.framework == "tf":
        #     pass
        return {"generated_sequence": generated_sequence, "input_ids": input_ids, "prompt_text": prompt_text}  #, "forward_res": forward_res}

  def forward(self, model_inputs, **forward_params):
        with self.device_placement():
            if self.framework == "tf":
                model_inputs["training"] = False
                model_outputs = self._forward(model_inputs, **forward_params)
            elif self.framework == "pt":
                inference_context = self.get_inference_context()
                with inference_context():
                    model_inputs = self._ensure_tensor_on_device(model_inputs, device=self.device)
                    model_outputs = self._forward(model_inputs, **forward_params)
                    model_outputs = self._ensure_tensor_on_device(model_outputs, device=torch.device("cpu"))
            else:
                raise ValueError(f"Framework {self.framework} is not supported")
        return model_outputs

  def postprocess(self, model_outputs, return_type=ReturnType.FULL_TEXT, clean_up_tokenization_spaces=True):
        generated_sequence = model_outputs["generated_sequence"][0]
        input_ids = model_outputs["input_ids"]
        prompt_text = model_outputs["prompt_text"]
        generated_sequence = generated_sequence.numpy().tolist()
        records = []
        for sequence in generated_sequence:
            if return_type == ReturnType.TENSORS:
                record = {"generated_token_ids": sequence}
            elif return_type in {ReturnType.NEW_TEXT, ReturnType.FULL_TEXT}:
                # Decode text
                text = self.tokenizer.decode(
                    sequence,
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=clean_up_tokenization_spaces,
                )

                # Remove PADDING prompt of the sequence if XLNet or Transfo-XL model is used
                if input_ids is None:
                    prompt_length = 0
                else:
                    prompt_length = len(
                        self.tokenizer.decode(
                            input_ids[0],
                            skip_special_tokens=True,
                            clean_up_tokenization_spaces=clean_up_tokenization_spaces,
                        )
                    )

                all_text = text[prompt_length:]
                if return_type == ReturnType.FULL_TEXT:
                    if isinstance(prompt_text, str):
                        all_text = prompt_text + all_text
                    elif isinstance(prompt_text, Chat):
                        # Explicit list parsing is necessary for parsing chat datasets
                        all_text = list(prompt_text.messages) + [{"role": "assistant", "content": all_text}]

                record = {"generated_text": all_text}
            records.append(record)
        return model_outputs, records



In [ ]:
prefix = "The recent advances in computational biology are"
access_token = ""
model = "meta-llama/Llama-2-7b-chat-hf"
# # model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model, token=access_token)

model = AutoModelForCausalLM.from_pretrained(
    model,
    token=access_token
 )



tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:

pipeline = transformers.pipeline(
"text-generation",
model=model,
tokenizer=tokenizer,
torch_dtype=torch.float16,
device=device,
pipeline_class=OurPipeline,
)

In [ ]:
# all_words = list(tokenizer.vocab.keys())
# prompts = []
# for _ in range(200):
#   prompts.append(' '.join([w for w in random.sample(dictionary, 10)]))

# print("prompt:", prompt)
sequences = pipeline(
'what is your name?\n',
do_sample=True,
top_k=10,
num_return_sequences=1,
eos_token_id=tokenizer.eos_token_id,
max_length=400,
)

# for seq in sequences:
#   print(f"{seq['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
[t.max() for t in sequences[0]['generated_sequence']['scores']]

[tensor(14.3316),
 tensor(15.5989),
 tensor(34.6459),
 tensor(41.6779),
 tensor(21.0846),
 tensor(28.7130),
 tensor(40.6874),
 tensor(39.9600),
 tensor(36.2525),
 tensor(33.0680),
 tensor(38.6480),
 tensor(22.0972),
 tensor(29.9447),
 tensor(38.4501),
 tensor(26.3731),
 tensor(43.3072),
 tensor(42.9244),
 tensor(45.8838),
 tensor(43.8740),
 tensor(39.5973),
 tensor(41.3439),
 tensor(33.7352),
 tensor(50.2644),
 tensor(43.6081),
 tensor(55.1568),
 tensor(44.0062),
 tensor(44.4625),
 tensor(46.2211),
 tensor(31.6713),
 tensor(29.9857),
 tensor(40.5708),
 tensor(23.4031),
 tensor(27.2269),
 tensor(43.7198),
 tensor(48.7022),
 tensor(47.2582),
 tensor(46.7363),
 tensor(48.4323),
 tensor(31.7695),
 tensor(32.0415),
 tensor(34.2337),
 tensor(28.8118),
 tensor(45.5339),
 tensor(33.1972),
 tensor(45.6103),
 tensor(48.1493),
 tensor(41.4282),
 tensor(28.5870),
 tensor(35.8872),
 tensor(32.0488),
 tensor(39.5841),
 tensor(37.7329),
 tensor(37.9567),
 tensor(46.0578),
 tensor(45.7994),
 tensor(43